<a href="https://colab.research.google.com/github/UBDBD/Deep-Learning_Project/blob/main/%ED%95%B4%EC%99%B8_%EA%B4%80%EA%B4%91%EC%A7%80%EB%A5%BC_%EC%9C%84%ED%95%9C_%ED%95%9C%EA%B8%80_%EB%82%9C%EB%8F%85%ED%99%94_%EB%AC%B8%EC%9E%A5_%EB%B3%B5%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정

In [ ]:
!pip install torch transformers datasets
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/scarletcho/KoG2P.git
!pip install g2pk

Cloning into 'KoG2P'...
remote: Enumerating objects: 65, done.
remote: Total 65 (delta 0), reused 0 (delta 0), pack-reused 65 (from 1)
Receiving objects: 100% (65/65), 33.03 KiB | 536.00 KiB/s, done.
Resolving deltas: 100% (30/30), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.9/580.9 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 69.1 MB/s eta 0:00:00


In [ ]:
# 라이브러리
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import time

from g2pk import G2p
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import Dataset as HFDataset
from multiprocessing import Pool
from torch.cuda.amp import autocast

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# G2P (데이터 전처리)

In [ ]:
# g2p 한글만 변환
class CustomG2p(G2p):
    def g2p_only_korean(self, text):
        return "".join([self(char) if re.fullmatch(r'[가-힣]', char) else char for char in text])

In [ ]:
# g2p 처리 함수
def g2p_process(text):
  g2p = CustomG2p()

  words = text.split(' ')
  g2p_word = [g2p.g2p_only_korean(word) for word in words]
  g2p_text = ' '.join(g2p_word)
  return g2p_text

In [ ]:
'''
# 학습 데이터 전처리
data = pd.read_csv('/content/drive/MyDrive/프로젝트/Deep-Learning/난독화된 한글 리뷰 복원_data/train.csv')

data_dict = {'input': data['input'].tolist(), 'output: data['output'].tolist()}

for i in tqdm(range(len(data['input']))):
    input_text = data['input'][i]
    g2p_text = g2p_process(input_text)

    data_dict['input'][i] = g2p_text
'''

"\n# 학습 데이터 전처리\ndata = pd.read_csv('/content/drive/MyDrive/프로젝트/Deep-Learning/난독화된 한글 리뷰 복원_data/train.csv')\n\ndata_dict = {'input': data['input'].tolist(), 'output: data['output'].tolist()}\n\nfor i in tqdm(range(len(data['input']))):\n    input_text = data['input'][i]\n    g2p_text = g2p_process(input_text)\n\n    data_dict['input'][i] = g2p_text\n"

In [ ]:
'''
# 전처리 저장
df = pd.DataFrame(data_dict)
df.to_csv('/content/drive/MyDrive/프로젝트/Deep-Learning/난독화된 한글 리뷰 복원_data/g2p_data.csv', encoding='utf-8-sig')
'''

"\n# 전처리 저장\ndf = pd.DataFrame(data_dict)\ndf.to_csv('/content/drive/MyDrive/프로젝트/Deep-Learning/난독화된 한글 리뷰 복원_data/g2p_data.csv', encoding='utf-8-sig')\n"

In [ ]:
# 데이터 불러오기
g2p_data = pd.read_csv('/content/drive/MyDrive/프로젝트/Deep-Learning/난독화된 한글 리뷰 복원_data/g2p_data.csv')
train_data, test_data = train_test_split(g2p_data, test_size=0.2, random_state=42)

# LSTM (음절 단위)

In [ ]:
# 음절 분리 함수
def split_syllables(text):
    return list(text)

In [ ]:
# 학습 데이터
data_lstm = train_data

In [ ]:
# 데이터 변환
data_lstm['input_syllables'] = data_lstm['input'].apply(split_syllables)
data_lstm['output_syllables'] = data_lstm['output'].apply(split_syllables)

In [ ]:
# 데이터 사전
char2index = defaultdict(lambda: len(char2index) + 1)
index2char = {}

if ' ' not in char2index:
    char2index[' '] = 1
    index2char[1] = ' '

for text in pd.concat([data_lstm['input_syllables'], data_lstm['output_syllables']]):
    for char in text:
        if char not in char2index:
            index = len(char2index) + 1
            char2index[char] = index
            index2char[index] = char

In [ ]:
# 데이터셋 정의
class SyllableDataset(Dataset):
    def __init__(self, data):
        self.inputs = data['input_syllables'].tolist()
        self.targets = data['output_syllables'].tolist()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        if isinstance(idx, list):
            return [self.__getitem__(i) for i in idx]

        input_seq = self.inputs[idx]
        target_seq = self.targets[idx]

        input_tensor = torch.tensor([char2index.get(ch, char2index[' ']) for ch in input_seq], dtype=torch.long)
        target_tensor = torch.tensor([char2index.get(ch, char2index[' ']) for ch in target_seq], dtype=torch.long)

        return input_tensor, target_tensor


In [ ]:
# 데이터 패딩 함수
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded.to(device), targets_padded.to(device)

In [ ]:
# LSTM 모델 정의 (Bidirectional)
class SyllableLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(SyllableLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)
        return self.softmax(output)

In [ ]:
# 데이터셋 변환
batch_size = 64

train_data_lstm = train_data.reset_index(drop=True)
train_dataset_lstm = SyllableDataset(train_data)
train_loader_lstm = DataLoader(train_dataset_lstm, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [ ]:
# 학습 설정
vocab_size = len(char2index) + 1
embedding_dim = 128
hidden_dim = 256
num_layers = 2
num_epochs = 10

lstm_model = SyllableLSTM(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001, weight_decay=1e-5)

In [ ]:
# 모델 학습
'''
torch.cuda.empty_cache()

for epoch in range(num_epochs):
    total_loss = 0
    for input_batch, target_batch in tqdm(train_loader_lstm):
        optimizer.zero_grad()
        output = lstm_model(input_batch)

        # 길이 맞춰서 자르기
        min_len = min(output.size(1), target_batch.size(1))
        output = output[:, :min_len, :].contiguous()
        target_batch = target_batch[:, :min_len].contiguous()

        loss = criterion(output.reshape(-1, vocab_size), target_batch.reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss:.2f}')
'''

"\ntorch.cuda.empty_cache()\n\nfor epoch in range(num_epochs):\n    total_loss = 0\n    for input_batch, target_batch in tqdm(train_loader_lstm):\n        optimizer.zero_grad()\n        output = lstm_model(input_batch)\n\n        # 길이 맞춰서 자르기\n        min_len = min(output.size(1), target_batch.size(1))\n        output = output[:, :min_len, :].contiguous()\n        target_batch = target_batch[:, :min_len].contiguous()\n\n        loss = criterion(output.reshape(-1, vocab_size), target_batch.reshape(-1))\n        loss.backward()\n        optimizer.step()\n        total_loss += loss.item()\n\n    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss:.2f}')\n"

In [ ]:
# 모델 저장
'''
torch.save(lstm_model.state_dict(), '/content/drive/MyDrive/프로젝트/Deep-Learning/LSTM_model')
with open('/content/drive/MyDrive/프로젝트/Deep-Learning/char2index.pkl', 'wb') as f:
    pickle.dump(dict(char2index), f)
'''

"\ntorch.save(lstm_model.state_dict(), '/content/drive/MyDrive/프로젝트/Deep-Learning/LSTM_model')\nwith open('/content/drive/MyDrive/프로젝트/Deep-Learning/char2index.pkl', 'wb') as f:\n    pickle.dump(dict(char2index), f)\n"

In [ ]:
# 모델 불러오기
with open('/content/drive/MyDrive/프로젝트/Deep-Learning/char2index.pkl', 'rb') as f:
    char2index_loaded = pickle.load(f)

char2index = defaultdict(lambda: len(char2index_loaded) + 1)
char2index.update(char2index_loaded)

lstm_model = SyllableLSTM(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)
lstm_model.load_state_dict(torch.load('/content/drive/MyDrive/프로젝트/Deep-Learning/LSTM_model'))

<All keys matched successfully>

In [ ]:
# 복원 함수
def restored_lstm(model, input_text):
    model.eval()

    input_encoded = [char2index[char] if char in char2index else char2index[' '] for char in input_text]
    input_tensor = torch.tensor(input_encoded, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        predicted_indices = output.argmax(dim=2).squeeze(0).tolist()

    valid_indices = [idx if idx in index2char else 1 for idx in predicted_indices]
    restored_text = "".join([index2char[idx] for idx in valid_indices])

    if len(restored_text) < len(input_text):
        restored_text += ' ' * (len(input_text) - len(restored_text))
    elif len(restored_text) > len(input_text):
        restored_text = restored_text[:len(input_text)]

    return restored_text

In [ ]:
# 테스트
input_text = '녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜! ㅎㅎ 당음웨 또 옭 컷 갗았요.'

lstm_start_time = time.time()

g2p_text = g2p_process(input_text)
restored_text = restored_lstm(lstm_model, g2p_text)

lstm_end_time = time.time()
lstm_eval_time = lstm_end_time - lstm_start_time

print(f'input: {input_text}')
print(f'output: {restored_text}')
print(f'time: {lstm_eval_time}')

input: 녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜! ㅎㅎ 당음웨 또 옭 컷 갗았요.
output: 너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다! ㅎㅎ 다음에 또 올 것 같아요.
time: 2.1283371448516846


#KoBART (문장 단위)

In [ ]:
# 모델 / 토크나이저 로드
kobart_model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')
kobart_tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [ ]:
# 토크나이징 함수
def tokenize_function(example):
    return kobart_tokenizer(example['input'], text_target=example['output'], padding='max_length', truncation=True, max_length=512)

In [ ]:
# 학습 데이터
data_kobart = train_data

In [ ]:
# 문장 단위 분리
sentence_data = {'input': [], 'output': []}

for inp_text, out_text in zip(data_kobart['input'], data_kobart['output']):
    inp_sents = re.split(r'(?<=[.?!])\s+', inp_text)
    out_sents = re.split(r'(?<=[.?!])\s+', out_text)

    inp_sents = [s.strip() for s in inp_sents if s.strip()]
    out_sents = [s.strip() for s in out_sents if s.strip()]

    for inp, out in zip(inp_sents, out_sents):
        sentence_data['input'].append(inp)
        sentence_data['output'].append(out)


In [ ]:
# 데이터셋 변환
train_dataset_kobart = HFDataset.from_dict(sentence_data)
tokenized_datasets_kobart = train_dataset_kobart.map(tokenize_function, batched=True)

Map:   0%|          | 0/27725 [00:00<?, ? examples/s]

In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='no',
    save_strategy='epoch',
    per_device_train_batch_size=64,
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=500,
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=kobart_model,
    args=training_args,
    train_dataset=tokenized_datasets_kobart,
)

In [ ]:
# 모델 학습
'''
torch.cuda.empty_cache()

trainer.train() # key: bd51fc2a0c2d5b10846d860e631271c8ddc50515
'''

'\ntorch.cuda.empty_cache()\n\ntrainer.train() # key: bd51fc2a0c2d5b10846d860e631271c8ddc50515\n'

In [ ]:
# 모델 저장
'''
kobart_model.save_pretrained('/content/drive/MyDrive/프로젝트/Deep-Learning/KoBART_model')
kobart_tokenizer.save_pretrained('/content/drive/MyDrive/프로젝트/Deep-Learning/KoBART_tokenizer')
'''

"\nkobart_model.save_pretrained('/content/drive/MyDrive/프로젝트/Deep-Learning/KoBART_model')\nkobart_tokenizer.save_pretrained('/content/drive/MyDrive/프로젝트/Deep-Learning/KoBART_tokenizer')\n"

In [ ]:
# 모델 불러오기
kobart_model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/프로젝트/Deep-Learning/KoBART_model')
kobart_tokenizer = PreTrainedTokenizerFast.from_pretrained('/content/drive/MyDrive/프로젝트/Deep-Learning/KoBART_tokenizer')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [ ]:
def restored_kobart(model, tokenizer, text, max_output_len=512, batch_size=64):
    model.eval()

    sentences = re.split(r'(?<=[.?!])\s+', text.strip())
    if not sentences:
        return text

    restored_sentences = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        inputs = tokenizer(
            batch,
            return_tensors='pt',
            max_length=512,
            truncation=True,
            padding=True
        ).to(model.device)

        with torch.no_grad():
          with torch.amp.autocast('cuda'):
            generated_ids = model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=max_output_len,
                num_beams=1,
                early_stopping=True
            )

        decoded_sentences = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        restored_sentences.extend(decoded_sentences)

    return " ".join(restored_sentences)

In [ ]:
# 테스트
test_text = '녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜! ㅎㅎ 당음웨 또 옭 컷 갗았요.'

kobart_start_time = time.time()

g2p_text = g2p_process(test_text)
restored_text = restored_kobart(kobart_model, kobart_tokenizer, g2p_text)

kobart_end_time = time.time()
kobart_eval_time = kobart_end_time - kobart_start_time

print(f'input: {test_text}')
print(f'output: {restored_text}')
print(f'time: {kobart_eval_time}')

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


input: 녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜! ㅎㅎ 당음웨 또 옭 컷 갗았요.
output: 너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다! ᄒᄒ 다음에 또 올 것 같아요.
time: 12.80529499053955


# 평가

In [ ]:
# 평가 데이터
eval_data = test_data

In [ ]:
# 평가 함수
def evaluate_model(preds, targets):
    all_num_same, all_pred_len, all_target_len = 0, 0, 0

    for pred, target in zip(preds, targets):
        length = min(len(pred), len(target))
        num_same = sum([1 for i in range(length) if pred[i] == target[i]])

        all_num_same += num_same
        all_pred_len += len(pred)
        all_target_len += len(target)

    precision = all_num_same / all_pred_len if all_pred_len > 0 else 0.0
    recall = all_num_same / all_target_len if all_target_len > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

    return {
        'num_same': all_num_same,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

## 1. LSTM

In [ ]:
lstm_preds = []
lstm_targets = []

for _, row in tqdm(eval_data.iterrows(), total=len(eval_data)):
    input_text = row['input']
    target_text = row['output']

    lstm_out = restored_lstm(lstm_model, input_text)

    lstm_preds.append(lstm_out)
    lstm_targets.append(target_text)

lstm_scores = evaluate_model(lstm_preds, lstm_targets)

print('')
print(f'LSTM: {lstm_scores}')


100%|██████████| 2253/2253 [00:10<00:00, 223.82it/s]



LSTM: {'num_same': 199140, 'precision': 0.9485883591431552, 'recall': 0.9482631365919859, 'f1': 0.9484257199872361}


## 2. KoBART

In [ ]:
kobart_preds = []
kobart_targets = []

for _, row in tqdm(eval_data.iterrows(), total=len(eval_data)):
    input_text = row['input']
    target_text = row['output']

    kobart_out = restored_kobart(kobart_model, kobart_tokenizer, input_text)

    kobart_preds.append(kobart_out)
    kobart_targets.append(target_text)

kobart_scores = evaluate_model(kobart_preds, kobart_targets)

print('')
print(f'KoBART: {kobart_scores}')

100%|██████████| 2253/2253 [5:02:07<00:00,  8.05s/it]


KoBART: {'num_same': 96430, 'precision': 0.4600426504334219, 'recall': 0.4591795433442061, 'f1': 0.45961069168001223}


## 3. LSTM + KoBART

In [ ]:
lstm_preds = []
lsko_preds = []
lsko_targets = []

for _, row in tqdm(eval_data.iterrows(), total=len(eval_data)):
    input_text = row['input']
    target_text = row['output']

    lstm_out = restored_lstm(lstm_model, input_text)

    lstm_preds.append(lstm_out)
    lsko_targets.append(target_text)

for _, row in tqdm(lstm_preds.iterrows(), total=len(lstm_preds)):
    input_text = row['input']

    lsko_out = restored_kobart(kobart_model, kobart_tokenizer ,lstm_preds)

    lsko_preds.append(lsko_out)

lsko_scores = evaluate_model(lsko_out, lsko_targets)

print('')
print(f'lstm + kobart: {lsko_scores}')

# 추가 테스트

In [ ]:
print(lstm_targets[2])
print(lstm_preds[2])
print(kobart_targets[2])
print(kobart_preds[2])
print(lsko_targets[2])
print(lsko_preds[2])